In [1]:
import pandas as pd
import numpy as np

In [2]:
paths = ["C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/1_Eventos.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/2_MDB.xlsx",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/3_Dependientes.xlsx",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/4_edformal.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/5_Experiencia.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/6_idiomas.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/7_Intereses_carrera.csv",
         "C:/Users/JMoncada43/OneDrive - Cementos Argos S.A/Documentos/Python Scripts/Proyecto Retiros/data/8_software_skills.csv"
         ]

In [3]:
Eventos = pd.read_csv(paths[0], skiprows=1)
MDB = pd.read_excel(paths[1], skiprows=2)
Depends = pd.read_excel(paths[2], skiprows=2)
Education = pd.read_csv(paths[3], skiprows=1)
Exp = pd.read_csv(paths[4], skiprows=1)
Languages = pd.read_csv(paths[5], skiprows=1)
Interest = pd.read_csv(paths[6], skiprows=1)
Software = pd.read_csv(paths[7], skiprows=1)

In [96]:
def filter_events(filtered_df):
    # Filtrar registros según las condiciones dadas
    filtered_df = filtered_df.loc[
        (filtered_df['event'] == 'TERMINATION') &
        (filtered_df['eventReason'] == 'Retiro Voluntario')
    ]

    # Seleccionar columnas de interés
    columns_to_keep = ['eventCode', 'event', 'eventReasonCode', 'eventReason',
                       'effectiveDate', 'personId', 'termination detail']
    filtered_df = filtered_df[columns_to_keep]

    # Eliminar registros duplicados
    filtered_df = filtered_df.drop_duplicates()

    filtered_df['personId'] = filtered_df['personId'].astype(object)

    return filtered_df

In [97]:
def filter_depends(df):
    columns_to_keep = ['PERSON ID', 'DATE OF BIRTH','RELATIONSHIP']
    dfDep = df[columns_to_keep]

    # Eliminar los NaN
    dfDep.dropna(inplace = True)

    # Obtener conteo de dependientes de cada PersonId
    dfDep = dfDep.groupby('PERSON ID')['RELATIONSHIP'].count().reset_index()
    dfDep.rename(columns={'RELATIONSHIP': 'DEPENDIENTES'}, inplace=True)
    dfDep['PERSON ID'] = dfDep['PERSON ID'].astype(object)

    return dfDep

In [98]:
def filter_mdb(df):
    # Filtrar registros según las condiciones dadas
    dfMDB = df.loc[
        ((df['EMPLOYEE STATUS'] == 'TERMINATED') | (df['EMPLOYEE STATUS'] == 'ACTIVE'))
        ]

    # Eliminar registros duplicados
    dfMDB = dfMDB.drop_duplicates()

    dfMDB['PERSON ID'] = dfMDB['PERSON ID'].astype(object)

    return dfMDB

In [99]:
def filter_exp(df):

    # Quitar fechas erradas, volverlas NaN (ej: año 1810)
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
    df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')

    # Eliminar los NaN
    df.dropna(subset=['Start Date', 'End Date'], inplace=True)

    # Convertir en formato fecha
    df['Start Date']  = pd.to_datetime(df['Start Date'])
    df['End Date']  = pd.to_datetime(df['End Date'])

    # Obtener años de permanencia en cada experiencia
    df['Years_Exp'] = (df['End Date'] - df['Start Date']).dt.days / 365

    # Obtener años totales de experiencia de cada colaborador
    dfExp = df.groupby('Person ID External')['Years_Exp'].sum().reset_index()

    dfExp['Person ID External'] = dfExp['Person ID External'].astype(object)

    return dfExp

In [100]:
def filter_education (df):

    # Seleccionar las variables de interes
    dfEducation = df[['Person ID External', 'Academic degree']]

    # Obtener años de educacion
    dfYears = df[['Person ID External', 'End year', 'Start year']]
    dfYears['Years_Educ'] = dfYears['End year'] - dfYears['Start year']
    dfYears = dfYears.groupby('Person ID External')['Years_Educ'].sum().reset_index()


    # Dicotomizar variables
    dfEducation = pd.get_dummies(dfEducation, columns=['Academic degree'], drop_first=True, prefix=['Academic degree'])

    # Renombrar columnas
    dfEducation.rename(columns={"Person ID External": 'PersonID', 
                          "Academic degree_BACHELOR'S": "Bachelor",
                          "Academic degree_DOCTORATE'S": "Doctorate", 
                          "Academic degree_ELEMENTARY SCHOOL": "Elementary_school",
                          "Academic degree_HIGH SCHOOL": "High_school", 
                          "Academic degree_MASTER'S": "Master",
                          "Academic degree_SPECIALIZATION (COL)": "Specialization",
                          "Academic degree_TECHNICAL (COL)": "Technical"}, inplace=True)
    

    # Quitar duplicados por medio de agrupación
    dfEducation = dfEducation.groupby("PersonID").any().reset_index()

    dfEducation = dfEducation.merge(dfYears, how='left', left_on='PersonID' , right_on='Person ID External')
    dfEducation.drop(columns=['Person ID External'], inplace=True)

    dfEducation['PersonID'] = dfEducation['PersonID'].astype(object)    

    return dfEducation

In [101]:
def filter_lang(df):

    # Seleccionar variables de interés
    df_Lang = df[['Person ID External', 'Language/Dialect']]

    # Dicotomizar variables
    df_Lang = pd.get_dummies(df_Lang, columns=['Language/Dialect'], drop_first=True, prefix=['Language/Dialect'])

    # Sumar todas las columnas de lenguaje por colaborador (Person ID External)
    df_Lang['Languages'] = df_Lang.iloc[:, 1:].sum(axis=1)

    # Quitar las columnas de lenguaje individuales
    df_Lang = df_Lang[['Person ID External', 'Languages']]

    # Sumar los valores de 'Languages' por 'Person ID External'
    df_Lang = df_Lang.groupby('Person ID External')['Languages'].sum().reset_index()

    df_Lang['Person ID External'] = df_Lang['Person ID External'].astype(object)

    return df_Lang

In [102]:
def filter_interest (df):

    # Seleccionar variables de interés
    dfInter = df[['Person ID External', 'Department/Team interested in']]

    # Eliminar registros que no tienen un area de interes
    dfInter = dfInter.dropna(subset=['Department/Team interested in'])

    # Agrupar todas las areas de interes en un solo registro
    dfInter = dfInter.groupby("Person ID External")["Department/Team interested in"].apply(lambda x: " / ".join(x)).reset_index()

    dfInter['Person ID External'] = dfInter['Person ID External'].astype(object)
    return dfInter

In [104]:
def filter_software (df):

    # Seleccionar variables de interés
    dfSoft = df[['Person ID External', 'Application']]

    # Eliminar registros que no tienen una skill
    dfSoft = dfSoft.dropna(subset=['Application'])

    # Agrupar todos los skills en un solo registro
    dfSoft = dfSoft.groupby("Person ID External")["Application"].apply(lambda x: " / ".join(x)).reset_index()



    # Seleccionar variables de interés
    df_count = df[['Person ID External', 'Application']]

    # Dicotomizar variables
    df_count = pd.get_dummies(df_count, columns=['Application'], drop_first=True, prefix=['Application'])

    # Sumar todas las columnas de lenguaje por colaborador (Person ID External)
    df_count['Count_App'] = df_count.iloc[:, 1:].sum(axis=1)

    # Quitar las columnas de lenguaje individuales
    df_count = df_count[['Person ID External', 'Count_App']]

    # Sumar los valores de 'Count_App' por 'Person ID External'
    df_count = df_count.groupby('Person ID External')['Count_App'].sum().reset_index()

    dfSoft = dfSoft.merge(df_count, how = 'left', on = 'Person ID External')

    dfSoft['Person ID External'] = dfSoft['Person ID External'].astype(object)

    return dfSoft

In [121]:
# Unir todos los df

def concat(df1, df2, df3, df4, df5, df6, df7, df8):

    """
    df1: MDB
    df2: Eventos
    df3: Dependientes
    df4: Exp. Laboral
    df5: Educación
    df6: Idiomas
    df7: Intereses laborales
    df8: Software
    """
    df = df1.merge(df2, how='left', left_on="PERSON ID", right_on="personId", suffixes=('_df1', '_df2'))
    df = df.merge(df3, how='left', left_on="PERSON ID", right_on="PERSON ID", suffixes=('', '_df3'))
    df = df.merge(df4, how='left', left_on="PERSON ID", right_on="Person ID External", suffixes=('', '_df4'))
    df = df.merge(df5, how='left', left_on="PERSON ID", right_on="PersonID", suffixes=('', '_df5'))
    df = df.merge(df6, how='left', left_on="PERSON ID", right_on="Person ID External", suffixes=('', '_df6'))
    df = df.merge(df7, how='left', left_on="PERSON ID", right_on="Person ID External", suffixes=('', '_df7'))
    df = df.merge(df8, how='left', left_on="PERSON ID", right_on="Person ID External", suffixes=('', '_df8'))

    df.drop(columns=[
        'CURRENCY', 'FREQ', 'CURRENCY.1', 'FREQ.1', 'CURRENCY.2', 'FREQ.2', 'CURRENCY.3', 'FREQ.3', 'CURRENCY.4', 'FREQ.4', 'CURRENCY.5', 'FREQ.5', 
        'CURRENCY.6', 'FREQ.6', 'CURRENCY.7', 'FREQ.7', 'CURRENCY.8', 'FREQ.8', 'CURRENCY.9', 'FREQ.9', '(NO ILA) PLAN VEHICULAR', '(NO ILA) PAGO PERSONAL',
        'BONO DE AHORRO RD', 'G DE REP', 'PRIMA HAITI', 'HOURLY', 'BASE SALARY','personId', 'Person ID External', 
        'PersonID', 'Person ID External_df6', 'Person ID External_df7', 'Person ID External_df8'
    ], inplace=True)
    
    return df

In [122]:
# Generar base consolidada

df = concat(
    filter_mdb(MDB), 
    filter_events(Eventos),
    filter_depends(Depends),
    filter_exp(Exp),
    filter_education(Education),
    filter_lang(Languages),
    filter_interest(Interest),
    filter_software(Software)     
)

C:\Users\JMoncada43\AppData\Local\Temp\ipykernel_2676\816401983.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDep.dropna(inplace = True)
C:\Users\JMoncada43\AppData\Local\Temp\ipykernel_2676\4042779431.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfYears['Years_Educ'] = dfYears['End year'] - dfYears['Start year']


In [123]:
df

,POS CODE,POSITION,POS EMP GROUP,COUNTRY,COMPANY CODE,COMPANY,ID FISCAL,BUSINESS UNIT CODE,BUSINESS UNIT,DIVISION CODE,DIVISION,DEPARTMENT CODE,DEPARTMENT,GROUPING PROCESS,COST CENTER CODE,COST CENTER,JOB CODE,JOB,GRADE,SALARY GRADE,JOB LEVEL,PAY GRADE LEVEL,PAY SCALE AREA,PAY SCALE TYPE,PAY SCALE GROUP,PAY SCALE LEVEL,UNIONIOSED,PARENT POSITION CODE,PARENT POSITION,HR MANAGER POS CODE,HR MANAGER POSITION,MATRIX MANAGER POS CODE,MATRIX MANAGER POSITION,PERSON ID,USER ID,EMPLOYEE STATUS,EMPLOYEE GROUP,EMPLOYEE SUBGROUP,EMP DATE OF BIRTH,EMP HIRE DATE,EMP ORIGINAL START DATE,CONTRACT TYPE,CONTRACT END DATE,WORK RELATIONSHIP,BENEFITS TYPE,UNION 1,UNION 2,TEMP COMPANY,EMP GENDER,EMP MARITAL STATUS,LOCATION CODE,LOCATION,LOCATION GROUP,SUPERVISOR PERSON ID,SUPERVISOR POS CODE,SUPERVISOR POSITION,ORDINARY SALARY (COL),INTEGRAL SALARY (COL),SUPPORT PAYMENT (COL),MONTHLY AMOUNT,ANNUAL SALARY,eventCode,event,eventReasonCode,eventReason,effectiveDate,termination detail,DEPENDIENTES,Years_Exp,Bachelor,Doctorate,Elementary_school,High_school,Master,Specialization,Technical,Years_Educ,Languages,Department/Team interested in,Application,Count_App
0,CEM105110,CONDUCTOR(A) MIXER,EMPLOYEE,COL,CEM-CO05,CONCRETOS ARGOS S.A.,860350697,CEM80000003,VICEPRESIDENCIA REGIONAL COLOMBIA,CEM70000048,GERENCIA NEGOCIO INDUSTRIAL,CEM20000525,JEFATURA PLANTA SAN MARCOS,CONCRETE,1000CO054040R0,DIST CONC PTO TEJADA,CEM40000207,CONDUCTOR(A) MIXER,GRADO 00,0,TECHNICAL /OPERATIVES,6,CONCRETOS ARGOS S.A.,CONVENCION,CONV_CON_COL,NIVEL 05 CON,True,CEM108191,JEFE(A) PLANTA CONCRETO,CEM100958,COORDINADOR(A) GESTION HUMANA Y ADTVA.,NaN,NaN,2000012212,NaN,TERMINATED,EMPLOYEE,OPERATIVE,1970-12-22,2014-01-21,2014-01-21,FIXED TERM,2018-01-20,ORDINARY / ACTUAL,NaN,NaN,NaN,NaN,M,COHABITATION,C050,P PTO TEJADA,PLANTAS DE CONCRETO FIJAS,2.000004e+09,CEM108191,JEFE(A) PLANTA CONCRETO,1605901.0,NaN,NaN,1605901.00,2.401357e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CEM121294,DRIVER READY MIX TRUCK,EMPLOYEE,USA,CEM-US02,ARGOS USA LLC,452429726,CEM80000001,USA PRESIDENCY,CEM70000090,RMX BUSINESS UNIT,CEM20001110,PLANT SUPERVISION NE SOUTH SARASOTA,CONCRETE,1000US026436R0,SOUTH SARASOTA_DISTR,CEM40001317,DRIVER READY MIX TRUCK,GRADO 00,0,TECHNICAL /OPERATIVES,6,HOURLY,HOURLY,HOURLY_CEM_USA,HOURLY,False,CEM110892,PLANT SUPERVISOR NE,CEM110717,PEOPLE & CULTURE MANAGER,NaN,NaN,2000029751,NaN,TERMINATED,EMPLOYEE,HOURLY-PAID EMPLOYEE,1967-11-18,2021-12-13,2021-12-13,INDEFINITE DURATION CONTRACT,NaT,FULL-TIME REGULAR,BENEFITS PLAN,NaN,NaN,NaN,M,SINGLE,U411,SOUTH SARASOTA,PLANTAS DE CONCRETO FIJAS,2.000007e+09,CEM110892,PLANT SUPERVISOR NE,NaN,NaN,NaN,18.50,3.848000e+04,26,TERMINATION,205.0,Retiro Voluntario,12/18/2021,JOB DISSATISFACTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CEM113618,DRIVER READY MIX TRUCK,EMPLOYEE,USA,CEM-US02,ARGOS USA LLC,452429726,CEM80000001,USA PRESIDENCY,CEM70000090,RMX BUSINESS UNIT,CEM20003146,PLANT SUPERVISION OLD MILL,CONCRETE,1000US026198R0,ROCKY MOUNT_DISTRIB,CEM40001317,DRIVER READY MIX TRUCK,GRADO 00,0,TECHNICAL /OPERATIVES,6,HOURLY,HOURLY,HOURLY_CEM_USA,HOURLY,False,CEM113550,PLANT SUPERVISOR II,CEM112770,PEOPLE & CULTURE MANAGER,NaN,NaN,2000021558,NaN,TERMINATED,EMPLOYEE,HOURLY-PAID EMPLOYEE,1965-02-21,2018-07-23,2018-07-23,INDEFINITE DURATION CONTRACT,NaT,FULL-TIME REGULAR,BENEFITS PLAN,NaN,NaN,NaN,M,MARRIED,U180,ROCKY MOUNT,PLANTAS DE CONCRETO FIJAS,2.000010e+09,CEM113550,PLANT SUPERVISOR II,NaN,NaN,NaN,17.50,3.640000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CEM109892,DRIVER READY MIX TRUCK,EMPLOYEE,USA,CEM-US02,ARGOS USA LLC,452429726,CEM80000001,USA PRESIDENCY,CEM70000090,RMX BUSINESS UNIT,CEM20003358,PLANT MANAGEMENT,CONCRETE,1000US026117R0,ARLINGT NYA_DISTRIB,CEM40001317,DRIVER READY MIX TRUCK,GRADO 00,0,TECHNICAL /OPERATIVES,6,HOURLY,HOURLY,HOURLY_CEM_USA,HOURLY,False,CEM120714,PLANT MANAGER RM,CEM107946,PEOPLE & CULTURE MANAGER,NaN,NaN,2000

In [125]:
df.to_csv('BD.csv', index = False)
df.to_excel('BD.xlsx', index = False)